In [ ]:
!pip install --upgrade pip
# Installs the wheel compatible with CUDA 11 and cuDNN 8.2 or newer.
# Note: wheels only available on linux.
!pip install --upgrade "jax[cuda]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

In [ ]:
import jax
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random

from tensorflow import keras
import tensorflow_datasets as tfds
import tensorflow as tf

import time

## MLP by JAX

In [ ]:
def random_layer_params(m, n, key, scale=1e-2):
  w_key, b_key = random.split(key)
  return scale * random.normal(w_key, (n, m)), scale * random.normal(b_key, (n,))

In [ ]:
def init_network_params(layer_sizes, key):
  keys = random.split(key, len(layer_sizes))
  return [random_layer_params(m, n, k) for m, n, k in zip(layer_sizes[:-1], layer_sizes[1:], keys)]

In [ ]:
layer_sizes = [784, 512, 512, 10]
step_size = 0.01
num_epochs = 10
batch_size = 128
n_targets = 10
params = init_network_params(layer_sizes, random.PRNGKey(0))

In [ ]:
print(f'num. of layers: {len(params)}')
for i in range(len(params)):
  print(f'layer {i}: w={params[i][0].shape}, b={params[i][1].shape}')

In [ ]:
def relu(x):
  return jnp.maximum(0, x)

In [ ]:
from jax.scipy.special import logsumexp

In [ ]:
def predict(params, image):
  activations = image
  for w, b in params[:-1]:
    outputs = jnp.dot(w, activations) + b
    activations = relu(outputs)
  w_last, b_last = params[-1]
  logits = jnp.dot(w_last, activations) + b_last
  return logits - logsumexp(logits)

In [ ]:
random_flattened_image = random.normal(random.PRNGKey(1), (28 * 28,))
preds = predict(params, random_flattened_image)

In [ ]:
print(f'preds: {preds.shape}')
print(f'preds values: {preds}')

In [ ]:
batched_predict = vmap(predict, in_axes=(None, 0))

In [ ]:
random_flattened_images = random.normal(random.PRNGKey(1), (10, 28*28))
batched_preds = batched_predict(params, random_flattened_images)
print(batched_preds.shape)

In [ ]:
print(batched_preds[0, :])
print(jax.nn.softmax(batched_preds[0, :]))

In [ ]:
def one_hot(x, k, dtype=jnp.float32):
  # x[:, None] => add new axis (x.shape=(10,) -> x.shape=(10, 1))
  return jnp.array(x[:, None] == jnp.arange(k), dtype=dtype)

In [ ]:
def accuracy(params, images, targets):
  target_class = jnp.argmax(targets, axis=1)
  predicted_class = jnp.argmax(batched_predict(params, images), axis=1)
  return jnp.mean(predicted_class == target_class) # num. of True / target_class.shape[0] * target_class.shape[1]

In [ ]:
def loss(params, images, targets):
  preds = batched_predict(params, images)
  return -jnp.mean(preds * targets) # among the preds, only values with target class of 1 are considered.

In [ ]:
@jit
def update(params, x, y):
  grads = grad(loss)(params, x, y)
  return [(w - step_size * dw, b - step_size * db) for (w, b), (dw, db) in zip(params, grads)]

In [ ]:
data_dir = 'tfds_mnist'
mnist_data, info = tfds.load(name='mnist', batch_size=-1, data_dir=data_dir, with_info=True)

In [ ]:
mnist_data = tfds.as_numpy(mnist_data)

In [ ]:
train_data, test_data = mnist_data['train'], mnist_data['test']

In [ ]:
num_labels = info.features['label'].num_classes

In [ ]:
h, w, c = info.features['image'].shape

In [ ]:
num_pixels = h * w * c

In [ ]:
print(f'(h, w, c) = ({h}, {w}, {c})')
print(f'num_labels: {num_labels}')
print(f'train_data: {train_data["image"].shape}')
print(f'train_labels: {train_data["label"].shape}')
print(f'test_data: {test_data["image"].shape}')
print(f'test_labels: {test_data["label"].shape}')

In [ ]:
train_images, train_labels = train_data['image'], train_data['label']
train_images = jnp.reshape(train_images, (len(train_images), num_pixels))
train_labels = one_hot(train_labels, num_labels)

In [ ]:
test_images, test_labels = test_data['image'], test_data['label']
test_images = jnp.reshape(test_images, (len(test_images), num_pixels))
test_labels = one_hot(test_labels, num_labels)

In [ ]:
print(f'Train: {train_images.shape}, {train_labels.shape}')
print(f'Test: {test_images.shape}, {test_labels.shape}')

In [ ]:
def get_train_batches():
  ds = tfds.load(name='mnist', split='train', as_supervised=True, data_dir=data_dir)
  ds = ds.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
  return tfds.as_numpy(ds)

In [ ]:
for epoch in range(num_epochs):
  start_time = time.time()
  for x, y in get_train_batches():
    x = jnp.reshape(x, (len(x), num_pixels))
    y = one_hot(y, num_labels)
    params = update(params, x, y)
  epoch_time = time.time() - start_time

  train_acc = accuracy(params, train_images, train_labels)
  test_acc = accuracy(params, test_images, test_labels)
  print(f'Epoch {epoch} in {epoch_time:0.2f} sec')
  print(f'Train acc. {train_acc}')
  print(f'Test acc. {test_acc}')

## Data preparation

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

In [ ]:
print(f'x_train_shape: {x_train.shape} - y_train_shape: {y_train.shape}')
print(f'x_test_shape: {x_test.shape} - y_test_shape: {y_test.shape}')